In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np

test = np.load("/content/drive/MyDrive/python/done/competition-svhn/test.npy")
train = np.load("/content/drive/MyDrive/python/done/competition-svhn/train.npy")

test_im = test['images']
train_im = train['images']
train_lab = train['labels']

X = np.reshape(train_im, (-1, 32 * 32 * 3)) / 255.0
X = X.astype(np.float32)

y = np.zeros((train_lab.shape[0], 10), dtype=np.float32)
y[np.arange(len(y)), train_lab] = 1.0
#print(test_im.shape)
print(train_im.shape)
print(X.shape)
print(train_lab.shape)
print(y.shape)

In [ ]:
import tensorflow as tf
#optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

@tf.function
def cross_entropy(y_hat, y_true):
    return -tf.reduce_sum(y_true * tf.math.log(y_hat), axis=1)

class SoftmaxRegression:
    def __init__(self, learning_rate=0.1):
        self.learning_rate = learning_rate
        self.rng = np.random.default_rng(0)
        initial_values = self.rng.uniform(size=(3072, 10)).astype(np.float32)
        self.theta = tf.Variable(initial_values)

    @tf.function
#    def predict(self, X):
#        return tf.nn.softmax(tf.matmul(X, self.theta))
    def predict(self, X):
        logits = tf.matmul(X, self.theta)
        logits = logits - tf.reduce_max(logits, axis=1, keepdims=True)  # Stabilization trick
        return tf.nn.softmax(logits)

    @tf.function
    def training_step(self, X, y):
        with tf.GradientTape() as tape:
            y_hat = self.predict(X)
            loss = tf.reduce_mean(cross_entropy(y_hat, y))
        gradients = tape.gradient(loss, self.theta)
#        optimizer.apply_gradients(zip(gradients, [self.theta]))
        self.theta.assign_sub(self.learning_rate * gradients)

In [ ]:
class MiniBatchDataGenerator:
    def __init__(self, X, y, batch_size=100):
        self.X = X
        self.y = y
        self.batch_size = batch_size

        self.current_batch_idx = 0
        self.indices = np.arange(len(self.X))
        self.rng = np.random.default_rng(0)

    def __len__(self):
        return len(self.X) // self.batch_size

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_batch_idx == 0:
            self.rng.shuffle(self.indices)

        current_indices = self.indices[
            self.current_batch_idx * self.batch_size:
            (self.current_batch_idx + 1) * self.batch_size
        ]
        self.current_batch_idx += 1
        if self.current_batch_idx >= len(self):
            self.current_batch_idx = 0

        return self.X[current_indices], self.y[current_indices]

In [ ]:
model = SoftmaxRegression(learning_rate=0.01)
datagen = MiniBatchDataGenerator(X, y, batch_size=100)

epoch = 20

training_steps = len(datagen) * epoch
data_samples_involved = training_steps * datagen.batch_size
print(f"Epochs: {epoch}, training_steps: {training_steps}, data samples used: {data_samples_involved}")

for i in range(training_steps):
    X_batch, y_batch = next(datagen)
    model.training_step(X_batch, y_batch)

In [ ]:
import matplotlib.pyplot as plt

a=0

y_pred = model.predict(X)
#y_pred_int = np.argmax(y_pred, axis=1)
print(train_lab[a])
print(y[a])
print(train_im.shape)

np.set_printoptions(suppress=True)
print(y_pred[a].numpy())
#plt.imshow(train_im[a], cmap="gray")

In [ ]:
y_int = np.argmax(y_pred, axis=1)

accuracy = np.mean(y_int == train_lab)
print(f"Accuracy: {accuracy:.6f}")

In [ ]:
counts = np.bincount(train_lab)
most_frequent_value = np.argmax(counts)

print(f"Most frequently occuring category: {most_frequent_value}")
accuracy = np.mean(most_frequent_value == train_lab)
print(f"Accuracy: {accuracy:.6f}")

In [ ]:
data = np.array([y_int])

In [ ]:
import csv

with open("output.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data.tolist())  # Write multiple rows


In [ ]:
print(test_im.shape)
#print(X.shape)
X_test = np.reshape(test_im, (-1, 32 * 32 * 3)) / 255.0
X_test = X_test.astype(np.float32)

In [ ]:
a=5

test_pred = model.predict(X_test)
#y_pred_int = np.argmax(y_pred, axis=1)

np.set_printoptions(suppress=True)
print(y_pred[a].numpy())

In [ ]:
test_int = np.argmax(test_pred, axis=1)

In [ ]:
test_data = np.array([test_int])

In [ ]:
with open("test_output.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(test_data.tolist())  # Write multiple rows
